# Ubicación de las nuevas oficinas

### Mediante visualización de mapas de Cartoframes, consultas a MongoDB por coordenadas, y la ayuda de la API de Google Places, vamos a tratar de obtener la mejor ubicación para nuestra compañía de videojuegos

Importamos librerías

In [1]:
from pymongo import MongoClient
import pandas as pd
from src.gameslib import (geocode,gMapsLink,toCartoPoint,toGeoJSON,coordsFromAddress,tuplaFromPoint,withGeoQuery)
from src.gamesAPI import (numSchoolsFromCoords,numStarbucksFromCoords,numAirportsFromCoords,numClubsFromCoords)
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()
import re

Conectamos con la base de datos y a continuación usaremos la colección "cleancompanies"

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [3]:
empresas = list(db["cleancompanies"].find({}))
df = pd.DataFrame(empresas)
df.groupby(['category_code']).size().nlargest(10)

category_code
web                2307
software           1885
mobile              682
advertising         679
games_video         644
other               625
enterprise          542
consulting          445
ecommerce           412
network_hosting     410
dtype: int64

buscamos las empresas de videojuegos, diseño, etc

In [4]:

query = {"$or": [{"category_code":{'$regex':'/advertising|web|games_video|software/'}},{"tag_list":{'$regex':'/design|game|e-sport/'}}]}

designers = list(db["cleancompanies"].find(query))


In [5]:

df = pd.DataFrame(designers)
df.head()

,_id,name,category_code,number_of_employees,tag_list,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,geometry,location
0,5e41de06e5d83c5fe8cde474,Wetpaint,web,47.0,"wiki, seattle, elowitz, media-industry, media-...",,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,POINT (-73.9964312 40.7237306),"{'type': 'Point', 'coordinates': [-73.9964312,..."
1,5e41de06e5d83c5fe8cde476,Postini,web,5.0,,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573,POINT (-122.247573 37.506885),"{'type': 'Point', 'coordinates': [-122.247573,..."
2,5e41de06e5d83c5fe8cde479,Geni,web,18.0,"geni, geneology, social, family, genealogy",Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064,POINT (-118.393064 34.090368),"{'type': 'Point', 'coordinates': [-118.393064,..."
3,5e41de06e5d83c5fe8cde47a,Fox Interactive Media,web,0.0,,,407 N Maple Dr,,90210,Beverly Hills,CA,USA,34.076179,-118.394170,POINT (-118.39417 34.076179),"{'type': 'Point', 'coordinates': [-118.39417, ..."
4,5e41de06e5d83c5fe8cde47b,Flektor,games_video,5.0,"flektor, photo, video",None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768,POINT (-118.379768 34.025958),"{'type': 'Point', 'coordinates': [-118.379768,..."


### Visualizamos en unos mapas de Cartoframes la densidad de empresas del sector por zonas

In [7]:
#import geopandas as gpd

from cartoframes.viz import Map, Layer#, size_continuous_style,color_continuous_style
from cartoframes.viz.helpers import size_continuous_layer
from cartoframes.viz.widgets import histogram_widget

import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings('default')

In [35]:
from cartoframes.auth import set_default_credentials

set_default_credentials('cartoframes')

In [13]:
map_df = df[['geometry','number_of_employees']]
Map(Layer(map_df))#,geom_col='geometry'))

In [14]:
Map(size_continuous_layer(map_df,'number_of_employees'))

## Filtramos solo Europa

Creamos un polígono con Mongo Compass que uniremos a nuestra query de empresas del sector

In [15]:

poligono = {"location": {"$geoWithin": { "$geometry": { "type": 'Polygon', "coordinates": [ [ [ -15.307475904378945, 64.07403359101181 ], [ -15.659141724999325, 31.82118678722882 ], [ 52.56402747535335, 37.31096781038386 ], [ 50.98153128256165, 62.6576493310525 ], [ -15.307475904378945, 64.07403359101181 ] ] ] }}}}

european_designers = list(db["cleancompanies"].find( {"$and":[query,poligono]}))
df_eur = pd.DataFrame(european_designers)

In [16]:
Map(size_continuous_layer(df_eur[['geometry','number_of_employees']], 'number_of_employees'))

## Tomamos la decisión de analizar 4 ciudades centroeuropeas con mucha vinculación al mundo del videojuego, cerca de emplazamientos de grandes eventos.

In [21]:
candidatas = ['Paris','Bruselas','Amsterdam','Colonia']

cities = ['Paris,FR','Brussels,Belgium','Amsterdam','Colonia,Alemania']

Para operar tanto con la API de Google como con las geoqueries de MongoDB, tomamos las ubicaciones en varios formatos

In [22]:
points = [coordsFromAddress(city) for city in cities]
coords = [tuplaFromPoint(point) for point in points]
coords

[(48.856614, 2.3522219),
 (50.8503396, 4.3517103),
 (52.3666969, 4.8945398),
 (50.937531, 6.9602786)]

#### Escuelas de educación primaria:

In [23]:
schools = [numSchoolsFromCoords(coord) for coord in coords]

In [24]:
schools

[18, 5, 14, 1]

In [25]:
#clubs = [numClubsFromCoords(coord) for coord in coords]

#### Número de Starbucks cerca:

In [26]:
starbucks = [numStarbucksFromCoords(coord) for coord in coords]

In [27]:
starbucks

[9, 5, 7, 10]

#### Grandes aeropuertos cercanos

In [28]:
airports = [len(list(db["cleanairports"].find(withGeoQuery(point,20000)))) for point in points]

In [29]:
airports

[1, 1, 1, 1]

#### Empresas del sector en nuestra BBDD a menos de 5Km

In [30]:
empresas_cerca = [len(list(db["cleancompanies"].find({"$and":[query,withGeoQuery(point,5000)]}))) for point in points]

In [31]:
empresas_cerca

[48, 4, 17, 4]

#### Unimos las series

In [32]:
last_df = pd.DataFrame({"Ciudad":candidatas,
                        "Coordenadas":coords,
                        "Colegios":schools,
                        "Starbucks":starbucks,
                        "Aeropuertos":airports,
                        "Empresas":empresas_cerca})


#,"Night_clubs":clubs
# Pendiente de resolver el problema de la paginación para obtener más de 20 resultados

In [33]:
last_df

,Ciudad,Coordenadas,Colegios,Starbucks,Aeropuertos,Empresas
0,Paris,"(48.856614, 2.3522219)",18,9,1,48
1,Bruselas,"(50.8503396, 4.3517103)",5,5,1,4
2,Amsterdam,"(52.3666969, 4.8945398)",14,7,1,17
3,Colonia,"(50.937531, 6.9602786)",1,10,1,4


## La decisión se vuelve más clara con esta tabla resumen. ¡Nos vamos a París! 🇫🇷

Le pasamos a todos los empleados el enlace para que vayan conociendo su nuevo emplazamiento

In [34]:
print(gMapsLink(coordsFromAddress('Paris,France')))

https://www.google.com/maps/@48.856614,2.3522219,14z
